## USING TWELVE LABS API

In [12]:
import os
import requests
from pprint import pprint
from dotenv import load_dotenv

load_dotenv() #get environment variables from .env file

TL_API_URL = os.getenv('API_URL')
assert TL_API_URL

TL_API_KEY = os.getenv('API_KEY')
assert TL_API_KEY

headers = {
    "x-api-key": TL_API_KEY,
     "accept": "application/json",
    
}

### HELPER FUNCTIONS

In [31]:
import math

def convert_seconds(seconds:str):
    """Converts seconds to hours, minutes, and seconds in format HH:MM:SS or MM:SS"""
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds_remaining = math.floor(seconds % 60)
    
    if hours > 0:
        return f"{hours}:{minutes:02}:{seconds_remaining:02}"
    else:
        return f"{minutes}:{seconds_remaining:02}"
    

def extract_and_print_engine_info(data):
    """Extracts and prints engine information from JSON data."""
    sorted_data = sorted(data['data'], key=lambda x: x['_id'])  # Sort by engine ID
    
    for item in sorted_data:
        engine_id = item['_id']
        allowed_options = item['allowed_engine_options']
        ready = item['ready']

        print(f"Engine ID: {engine_id}")
        print(f"Allowed Options: {', '.join(allowed_options)}")
        print(f"Ready: {'Yes' if ready else 'No'}")
        print("="*40)


def extract_and_print_index_info(data):
    """This function extracts and prints index information from the given JSON data."""
    
    for item in data['data']:
        index_id = item['_id']
        index_name = item['index_name']
        created_at = item['created_at']
        engines = ", ".join([f"{engine['engine_name']} ({', '.join(engine['engine_options'])})" for engine in item['engines']])
        video_count = item['video_count']
        
        print(f"Index ID: {index_id}")
        print(f"Index Name: {index_name}")
        print(f"Created At: {created_at}")
        print(f"Engines: {engines}")
        print(f"Video Count: {video_count}")
        print("=" * 40)



def extract_and_print_video_info(data):
    """Extracts and prints video information."""
    for item in data['data']:
        video_id = item['_id']
        filename = item['metadata']['filename']
        duration = item['metadata']['duration']
        fps = item['metadata']['fps']
        width = item['metadata']['width']
        height = item['metadata']['height']

        resolution_str = f"{width} x {height}"

        print(f"Video ID: {video_id}")
        print(f"File Name: {filename}")
        print(f"Duration: {convert_seconds(duration)}")
        print(f"FPS: {fps}")
        print(f"Resolution: {resolution_str}")
        print("=" * 40)

def _get_video_name_from_video_id(index_id: str, video_id: str):
    """Get video information from index_id and video_id
    JSON Structure:
{
    '_id': 'some_video_id',
    'created_at': '2024-07-04T18:28:50Z',
    'hls': {
        'status': 'COMPLETE',
        'thumbnail_urls': ['something.jpg'],
        'updated_at': '2024-07-04T18:31:24.404Z',
        'video_url': 'something.m3u8'
    },
    'indexed_at': '2024-07-04T18:37:08Z',
    'metadata': {
        'duration': 600.766667,
        'engine_ids': ['pegasus1.1', 'marengo2.6'],
        'filename': 'plays.mp4',
        'fps': 30,
        'height': 360,
        'size': 55084753,
        'video_title': 'plays.mp4',
        'width': 640
    }
}
    """
    video_name_url = f"{TL_API_URL}/indexes/{index_id}/videos/{video_id}"
    response = requests.get(url=video_name_url, headers=headers)
    video_data = response.json()
    video_name = video_data['metadata']['filename']
    video_thumbnail = video_data['hls']['thumbnail_urls']
    video_url = video_data['hls']['video_url']
    
    
    return video_name, video_thumbnail, video_url
    
def extract_and_print_search_result_data(index_id: str, result_data):
    """This function takes in a JSON object of search results and displays relevant info.

    JSON structure:
    {
        'data': [
            {
                'confidence': 'high',
                'end': 572.5000000001367,
                'score': 85.57,
                'start': 511.933333333525,
                'video_id': 'SOME_VIDEO_ID'
            },
            {
                'confidence': 'high',
                'end': 479.700000000166,
                'score': 85.01,
                'start': 474.14166666682945,
                'video_id': 'SOME_VIDEO_ID'
            } ...
        ],

        'page_info': {
            'limit_per_page': 10,
            'next_page_token': '91f418f7-53be-4628-8daa-eec2c370d4ff-1',
            'page_expired_at': '2024-07-06T01:34:00Z',
            'total_results': 58
        },

        'search_pool': {
            'index_id': 'INDEX_ID',
            'total_count': 4,
            'total_duration': 2097
        }
    }

    Edit this function to add more info from the search result. 
    """
    
    result = result_data['data']
    
    if result:
        for item in result:
            video_id = item['video_id']
            video_name, video_thumbnail, video_url = _get_video_name_from_video_id(index_id=index_id, video_id=video_id)
            score = item['score']
            confidence = item['confidence']
            start = item['start']
            end = item['end']

            print(f"Video ID: {video_id}")
            print(f"Video Name: {video_name}")
            print(f"Video thumbnail: {video_thumbnail}")
            print(f"Video URL (HRL m3u8 format): {video_url}")
            print(f"Start: {convert_seconds(start)}")
            print(f"End: {convert_seconds(end)}")
            print(f"Score: {score}")
            print(f"Confidence: {confidence}")
            print("="*40)
    else:
        print("No relevant search result")



### List all engines

In [14]:
ENGINES_URL = f"{TL_API_URL}/engines"

response = requests.get(url=ENGINES_URL, headers=headers)

extract_and_print_engine_info(response.json())

Engine ID: marengo
Allowed Options: visual, text_in_video, conversation, logo
Ready: Yes
Engine ID: marengo2
Allowed Options: visual, text_in_video, conversation, logo
Ready: Yes
Engine ID: marengo2.5
Allowed Options: visual, text_in_video, conversation, logo
Ready: Yes
Engine ID: marengo2.6
Allowed Options: visual, text_in_video, conversation, logo
Ready: Yes
Engine ID: pegasus1
Allowed Options: visual, conversation
Ready: Yes
Engine ID: pegasus1.1
Allowed Options: visual+conversation, visual, conversation
Ready: Yes


### Create an Index

In [15]:
# INDEXES_URL = f"{TL_API_URL}/indexes"
# INDEX_NAME = "<MYINDEXNAME>" # Use a descriptive name for your index 
# data = {
#     "engine_id": "marengo",
#     "index_options": ["visual", "conversation", "text_in_video"],
#     "index_name": INDEX_NAME,
# }
# response = requests.post(INDEXES_URL, headers=headers, json=data)

# print (f'Status code: {response.status_code}')
# pprint (response.json())

### List all available index
This allows you to return a list of indexes in your account

In [16]:
INDEXES_URL = f"{TL_API_URL}/indexes"

response = requests.get(url=INDEXES_URL, headers=headers)

data = response.json()

extract_and_print_index_info(data)

Index ID: 6686e98c139f029878ae0e08
Index Name: 123
Created At: 2024-07-04T18:27:24.779Z
Engines: marengo2.6 (visual, conversation, text_in_video), pegasus1.1 (visual, conversation)
Video Count: 4
Index ID: 66844735eb3979a66bdc9f94
Index Name: IndexfromMacBook
Created At: 2024-07-02T18:30:13.506Z
Engines: pegasus1.1 (visual), marengo2.6 (visual)
Video Count: 3
Index ID: 66841d79eb3979a66bdc9f30
Index Name: Index1
Created At: 2024-07-02T15:32:09Z
Engines: marengo2.6 (visual, conversation, text_in_video, logo), pegasus1.1 (visual, conversation)
Video Count: 1
Index ID: 66841c757e3a05b242e8b122
Index Name: My Index (Default)
Created At: 2024-07-02T15:27:49.156Z
Engines: marengo2.6 (visual, conversation, text_in_video, logo), pegasus1.1 (visual, conversation)
Video Count: 0


In [17]:
INDEX_ID = "6686e98c139f029878ae0e08" #add valid index_id

### Upload a video

In [18]:
 #INDEX_TASK_URL = f"{TL_API_URL}/indexes/tasks"
# file_name = "<YOUR_FILE_NAME>" # Example: "test.mp4"
# file_path = "<YOUR_FILE_PATH>" # Example: "/Downloads/test.mp4"
# file_stream = open(file_path,'rb')
# data = {
#     "index_id": INDEX_ID, 
#     "language": "en"
# }
# file_param=[('video_file', (file_name, file_stream, 'application/octet-stream'))]
# response = requests.post(INDEX_TASK_URL, headers=headers, data=data, files=file_param)
# TASK_ID = response.json().get('_id')
# pprint (response.status_code)
# pprint (response.json())


### List all videos in Index

In [19]:
VIDEOS_URL = f"{TL_API_URL}/indexes/{INDEX_ID}/videos"

response = requests.get(url=VIDEOS_URL, headers=headers)

data = response.json()

extract_and_print_video_info(data)

Video ID: 6686eaead22b3a3c97bf3e28
File Name: Reporters Asking NBA Players Stupid Questions.mp4
Duration: 12:29
FPS: 30
Resolution: 640 x 360
Video ID: 6686e9e7d22b3a3c97bf3e27
File Name: LBJ.mp4
Duration: 4:18
FPS: 29.97
Resolution: 640 x 360
Video ID: 6686e9e5d22b3a3c97bf3e26
File Name: NBA Most Iconic Commentary Moments Of All Time.mp4
Duration: 8:07
FPS: 30
Resolution: 640 x 360
Video ID: 6686e9e2d22b3a3c97bf3e25
File Name: plays.mp4
Duration: 10:00
FPS: 30
Resolution: 640 x 360


### Make A Search Request

In [ ]:
SEARCH_QUERY = "Lebron James blocks shot"  # enter search prompt here

headers["Content-Type"] =  "multipart/form-data"
SEARCH_URL = f"{TL_API_URL}/search"

#read docs and manipulate as you wish
data = {
    "query": SEARCH_QUERY,
    "index_id": INDEX_ID,
    "search_options": ["visual", "conversation"], 
    "threshold" : "medium",
    "adjust_confidence_level": 0.55,
    "sort_option": "score",
    "conversation_option": "semantic",
    "page_limit": 10 #get only top 10 results
}
response = requests.post(SEARCH_URL, headers=headers, json=data)
result = response.json()
print(f"Searching '{SEARCH_QUERY}' with search options: {[opt for opt in data['search_options']]}\n")

if response.status_code == 200:
    extract_and_print_search_result_data(INDEX_ID, result)
else:
    pprint(result)
